In [120]:
from langchain.document_loaders import YoutubeLoader
from langchain.llms import OpenAI
from langchain.chains.summarize import load_summarize_chain
from langchain.text_splitter import RecursiveCharacterTextSplitter
import os
import ssl
ssl._create_default_https_context = ssl._create_unverified_context

In [121]:
my_key = os.getenv('OPENAI_API_KEY')

In [122]:
loader = YoutubeLoader.from_youtube_url("https://www.youtube.com/watch?v=-2v6EJE3XKo", add_video_info=True)

In [123]:
result = loader.load()

In [124]:
print (type(result))
print (f"Found video from {result[0].metadata['author']} that is {result[0].metadata['length']} seconds long")
print ("")
print (result)

<class 'list'>
Found video from Politics with Paint that is 1123 seconds long

[Document(page_content='The South China Sea: Arena, to what is one of the\xa0\nmost complex and dangerous territorial disputes\xa0\xa0 in the world. This incredibly valuable body of\xa0\nwater is defined by hundreds of tiny islands,\xa0\xa0 the ownership of which is highly contested between\xa0\nseveral nations. Vietnam, Malaysia, Brunei and the\xa0\xa0 Philippines all have overlapping claims here.\xa0\nBut the largest claim by far is made by China.\xa0\xa0 The infamous "Nine-dash Line" - also called the cow\'s\xa0\ntongue - claims about 90 percent of the South China\xa0\xa0 Sea for the People\'s Republic of China. And the\xa0\ncomplexity of this dispute does not end there.\xa0\xa0 Due to the South China Sea\'s wealth and\xa0\nnatural resources and its immense importance\xa0\xa0 as an international waterway, the conflict\xa0\nis also involving external powers like Japan,\xa0\xa0 India and most notably the Un

In [125]:
llm = OpenAI(temperature=0, openai_api_key=my_key)

In [126]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=2000, chunk_overlap=0)

In [131]:
texts = text_splitter.split_documents(result)
print(len(texts))

10


In [129]:
chain = load_summarize_chain(llm, chain_type="map_reduce", verbose=True)
chain.run(texts[:4])



> Entering new MapReduceDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
Write a concise summary of the following:


"The South China Sea: Arena, to what is one of the 
most complex and dangerous territorial disputes   in the world. This incredibly valuable body of 
water is defined by hundreds of tiny islands,   the ownership of which is highly contested between 
several nations. Vietnam, Malaysia, Brunei and the   Philippines all have overlapping claims here. 
But the largest claim by far is made by China.   The infamous "Nine-dash Line" - also called the cow's 
tongue - claims about 90 percent of the South China   Sea for the People's Republic of China. And the 
complexity of this dispute does not end there.   Due to the South China Sea's wealth and 
natural resources and its immense importance   as an international waterway, the conflict 
is also involving external powers like Japan,   India and most notably the United States, which 
directly ch

" The South China Sea is a heavily disputed region with multiple countries claiming ownership of hundreds of islands. The dispute involves external powers due to the region's valuable resources and strategic location. New technologies could make resource extraction easier, and fishing is important for the region's economy. The area is filled with small islands and reefs that are important for legal claims and military power. Six governments, including China and Taiwan, are involved in trying to control the region. In 2013, China's construction of artificial islands with military bases caused tension and international attention."

In [127]:
# chain = load_summarize_chain(llm, chain_type="stuff", verbose=False)
# chain.run(result)